In [1]:
# Author: M. Riley Owens (GitHub: mrileyowens)

# This file measures the spectroscopic 
# redshifts and properties of Lya profiles of 
# lensed LAE candidates identified in eBOSS by 
# Cao et al. 2020.

In [2]:
import os
import glob

import numpy as np

import pandas as pd

from astropy.io import fits

from scipy.optimize import curve_fit
from scipy.special import erf

import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText

from numba import njit

In [3]:
def extract_data(lae, home):

    '''
    Extracts data from the eBOSS fibers of the target

    Parameters:
        lae : str
            The SDSS ID of the target
        home : str
            The file path of this notebook

    Returns:
        w : numpy.ndarray
            Observed wavelength
        f : numpy.ndarray
            Observed flux density
        n : numpy.ndarray    
            Observed flux density error
    '''

    # Array containing the file path to the eBOSS fiber spectrum of the target
    fits_file = glob.glob(home + '/data/eBOSS_spectral_double_peaked_LAEs/*' + lae[0:9] + '*.fits')
            
    # The HDU list of the SDSS fiber spectrum
    hdul = fits.open(fits_file[0])
        
    # Get the observed wavelength, flux density, and flux density error
    # from extensions of the HDU list. The eBOSS fiber spectra are stored
    # with wavelength in log space and the error can be obtained from the
    # inverse variance
    w = 10**(hdul[1].data['loglam'])
    f = hdul[1].data['flux']
    n = 1 / np.sqrt(hdul[1].data['ivar'])

    return w, f, n

def compute_redshift(w, f, line_rest):

    '''
    Fits a Gaussian profile to an emission line and computes the object's
    redshift based on the emission line's rest wavelength

    Parameters:
        w : numpy.ndarray
            Observed wavelength
        f : numpy.ndarray
            Observed flux density
        line_rest : float
            Rest wavelength of the emission line

    Returns:
        p : numpy.ndarray
            Best-fit parameters of the Gaussian fit
        z : numpy.float64
            Redshift from the Gaussian fit
    '''

    # Gaussian function
    def gaussian(x, amp, cen, width):
        return amp * np.exp(-(x - cen)**2 / (2 * width**2))

    # Fit the emission line with a Gaussian function
    p, _ = curve_fit(gaussian, w, f, 
                     p0=(np.amax(f), np.mean(w), 5), 
                     bounds=([0, np.amin(w), 0], [2 * abs(np.amax(f)), np.amax(w), np.inf]), 
                     maxfev=1000)

    # The redshift based on the centroid of the Gaussian
    # fit and rest wavelength of the emission line
    z = (p[1] / line_rest) - 1

    return p, z

def compute_fcen(v, f):

    '''
    Computes the central escape fraction of Lya, as introduced by
    Naidu & Matthee et al. 2022.

    Parameters:
        v : numpy.ndarray    
            Peculiar velocity of Lya at the redshift of the LAE
        f : numpy.ndarray
            Flux density

    Returns:
        fcen : numpy.float64    
            Central escape fraction of Lya
    '''

    # Compute the central escape fraction of Lya
    fcen = np.trapz(f[(v >= -100) & (v <= 100)], v[(v >= -100) & (v <= 100)]) / np.trapz(f[(v >= -1000) & (v <= 1000)], v[(v >= -1000) & (v <= 1000)]) * 100

    return fcen

# Redundancy! v is an input but is not used
@njit
def compute_vsep(v, f_blue, f_red):

    '''
    Determines the peak separation between the redshifted and
    blueshifted Lya peaks.

    Parameters:
        v : numpy.ndarray
            Peculiar velocity of Lya at the redshift of the LAE
        f_blue : numpy.ndarray
            Flux density of the skewed Gaussian fit to the blueshifted peak
        f_red : numpy.ndarray
            Flux density of the skewed Gaussian fit to the redshifted peak

    Returns:
        vsep : numpy.int64
            Velocity separation between the maxima of the redshifted and blueshifted peaks
    '''
    
    # Compute the velocity separation between the maxima of the redshifted and blueshifted peaks
    vsep = abs(np.argmax(f_blue) - np.argmax(f_red))

    return vsep

def compute_fwhms(v, f_blue, f_red):

    '''
    Determines the FWHMs of the skewed Gaussian fits
    to the redshifted and blueshifted Lya peaks

    Parameters:
        v : numpy.ndarray
            Peculiar velocity of Lya at the redshift of the LAE
        f_blue : numpy.ndarray
            Flux density of the skewed Gaussian fit to the blueshifted peak
        f_red : numpy.ndarray
            Flux density of the skewed Gaussian fit to the redshifted peak

    Returns:    
        fwhm_b : numpy.float64
            FWHM of the skewed Gaussian fit to the blueshifted peak
        fwhm_r : numpy.float64
            FWHM of the skewed Gaussian fit to the redshifted peak
    '''

    def lin_interp(x, y, i, half):
        return x[i] + (x[i+1] - x[i]) * ((half - y[i]) / (y[i+1] - y[i]))

    v_new = np.arange(-1000,1000,1)

    fwhms = np.array([])

    # For each peak
    for i, peak in enumerate([f_blue, f_red]):

        # Interpolate the skewed Gaussian fit to the new velocity bins
        peak = np.interp(v_new, v, peak)

        # Half the flux density of the peak's maximum
        half = np.amax(peak) / 2

        # Determine the FWHM of the peak from linear interpolation
        signs = np.sign(peak - half)
        zero_crossings = (signs[0:-2] != signs[1:-1])
        zero_crossings_i = np.where(zero_crossings)[0]
        crossing1 = lin_interp(v_new, peak, zero_crossings_i[0], half)
        crossing2 = lin_interp(v_new, peak, zero_crossings_i[1], half)
        fwhm = abs(crossing1 - crossing2)

        fwhms = np.append(fwhms, fwhm)

        fwhm_b, fwhm_r = fwhms[0], fwhms[1]

    return fwhm_b, fwhm_r

def skewed_gaussian(x, amp, cen, width, skew):
    return amp * np.exp(-((x - cen) / width)**2 / 2) * (1 + erf(skew * ((x - cen) / width) / np.sqrt(2)))

#def double_skewed_gaussian(x, amp_b, cen_b, width_b, skew_b, amp_r, cen_r, width_r, skew_r):
#    return amp_b * np.exp(-((x - cen_b) / width_b)**2 / 2) * (1 + erf(skew_b * ((x - cen_b) / width_b) / np.sqrt(2))) + amp_r * np.exp(-((x - cen_r) / width_r)**2 / 2) * (1 + erf(skew_r * ((x - cen_r) / width_r) / np.sqrt(2)))

def fit_peaks(v, f):

    '''
    Fits two skewed Gaussians and a constant continuum level to the Lya profile
    
    Parameters:
        v : numpy.ndarray
            Peculiar velocity of Lya at the redshift of the LAE
        f : numpy.ndarray
            Flux density

    Returns:
        p : numpy.ndarray
            Best-fit values of the fit parameters
        cov : numpy.ndarray
            Covariance matrix of the parameters
    '''

    def double_skewed_gaussian(x, amp_b, cen_b, width_b, skew_b, amp_r, cen_r, width_r, skew_r, c):
        return amp_b * np.exp(-((x - cen_b) / width_b)**2 / 2) * (1 + erf(skew_b * ((x - cen_b) / width_b) / np.sqrt(2))) + amp_r * np.exp(-((x - cen_r) / width_r)**2 / 2) * (1 + erf(skew_r * ((x - cen_r) / width_r) / np.sqrt(2))) + c

    # Fit the Lya profile to two skewed Gaussians and a constant continuum
    p, cov = curve_fit(double_skewed_gaussian, v[(v >= -1000) & (v <= 1000)], f[(v >= -1000) & (v <= 1000)], 
        p0=(np.amax(f[(v >= -500) & (v <= 0)]), -250, 100, -1.0, np.amax(f[(v >= 0) & (v <= 1000)]), 250, 100, 1.0, np.nanmedian(f[(v >= 1000) & (v <= 2000)])), 
        bounds=([0, -1000, 0, -np.inf, 0, 0, 0, 0, np.amin(f[(v >= -2000) & (v <= 2000)])], [np.inf, 1000, np.inf, 0, np.inf, 1000, np.inf, np.inf, np.amax(f[(v >= -2000) & (v <= 2000)])]), maxfev=2000)

    return p, cov

def compute_lya_properties(lae_index, lines_to_use, windows):

    '''
    Computes various Lya parameters of the LAEs and their 
    spectroscopic redshift

    Parameters:
        lae_index :
            Index of LAE in the .csv catalog
        lines_to_use :
            Emission lines to use to measure the redshift
        windows :
            Wavelength bands around emission lines to fit
    '''

    # Set directories
    home = os.getcwd()
    figs = home + '/figs'

    # Open the double-peaked LAE catalog from Cao et al. 2020
    df = pd.read_csv(home + '/double_peak_catalog.csv', header=0)

    # Get the SDSS ID and initial redshift estimate of the LAE
    lae = df.iloc[:,0].to_numpy(dtype='str')[lae_index]
    z_init = df.iloc[:,4].to_numpy(dtype=np.float64)[lae_index]

    # Retrieve the eBOSS fiber spectrum of the target
    w, f, n = extract_data(lae, home)

    # Convert the wavelength to the peculiar velocity of Lya at the redshift of the LAE
    v_lya = 299792.458 * ((w / (1215.67 * (1 + z_init))) - 1)

    # Since we are only interested in analyzing the Lya profile, cut the eBOSS fiber spectrum to
    # -2100 < v < 2100 km/s about the Lya rest wavelength
    f_lya = f[(v_lya >= -2100) & (v_lya <= 2100)]
    n_lya = n[(v_lya >= -2100) & (v_lya <= 2100)]
    v_lya = v_lya[(v_lya >= -2100) & (v_lya <= 2100)]

    # Plot the LAE's Lya profile
    figLya, axLya = plt.subplots(figsize=(3,3), constrained_layout=True)
    
    axLya.set_xlim(-2000,2000)

    axLya.axvline(0, c='red', ls='dashed')
    axLya.plot(v_lya, f_lya, c='black', ds='steps-mid')
    axLya.plot(v_lya, n_lya, c='red', ds='steps-mid')

    plt.show()

    # Dictionary of common galaxy emsision lines (<2,000 Å; so eBOSS covers all lines 
    # at the redshift band of the LAEs), according to http://astronomy.nmsu.edu/drewski/tableofemissionlines.html
    line_dict = {'NV 1238' : 1238.821, 'NV 1242' : 1242.804, 'SiII 1260' : 1260.422,
        'SiII 1264' : 1264.730, 'OI 1302' : 1302.168, 'CII 1334' : 1334.532,
        'CII 1335' : 1335.708, 'SiIV 1393' : 1393.755, 'SiIV 1402' : 1402.770,
        'NIV] 1486' : 1486.496, 'CIV 1548' : 1548.187, 'CIV 1550' : 1550.772,
        'HeII 1640' : 1640.420, 'OIII] 1660' : 1660.809, 'OIII] 1666' : 1666.150, 
        'NIII] 1746' : 1746.823, 'NIII] 1748' : 1748.656, 'AlIII 1854' : 1854.716,
        'AlIII 1862' : 1862.790, 'SiIII] 1892' : 1892.030, '[CIII] 1906' : 1906.683, 
        'CIII] 1908' : 1908.734}

    # Create a diagnostic figure showing the spectrum at the position of common galaxy
    # emission lines at the redshift of the LAE
    figLines, axLines = plt.subplots(3,8, figsize=(16,6), constrained_layout=True)
    axLinesArr = np.array(axLines).reshape(-1)
 
    # There are more subplots than emission lines,
    # so turn off the extra ones
    axLines[2,6].axis('off')
    axLines[2,7].axis('off')

    line_to_use_index = 0

    for i, line in enumerate(line_dict):

        w_line = w[(w >= (line_dict[line] - 11) * (1 + z_init)) & (w <= (line_dict[line] + 11) * (1 + z_init))]
        f_line = f[(w >= (line_dict[line] - 11) * (1 + z_init)) & (w <= (line_dict[line] + 11) * (1 + z_init))]
        n_line = n[(w >= (line_dict[line] - 11) * (1 + z_init)) & (w <= (line_dict[line] + 11) * (1 + z_init))]

        axLinesArr[i].set_title(line)

        axLinesArr[i].set_xlim((line_dict[line] - 10) * (1 + z_init), (line_dict[line] + 10) * (1 + z_init))

        if line in lines_to_use:

            axLinesArr[i].set_title(line, color='red', fontweight='bold')

            axLinesArr[i].axvspan(windows[line_to_use_index][0], windows[line_to_use_index][1], color='gray', alpha=0.5)
                #w_line[(w_line >= windows[line_to_use_index][0]) & (w_line <= windows[line_to_use_index][1])])

            line_to_use_index = line_to_use_index + 1

        axLinesArr[i].axvline(line_dict[line] * (1 + z_init), c='red', ls='dashed')
        axLinesArr[i].plot(w_line, f_line, c='black', ds='steps-mid', lw=0.5)
        axLinesArr[i].plot(w_line, n_line, c='red', ds='steps-mid')

    plt.show()

    # Initialize the arrays that will store the MC measurements
    z_results = np.array([])
    fcen_results = np.array([])
    amp_b_results = np.array([])
    cen_b_results = np.array([])
    width_b_results = np.array([])
    skew_b_results = np.array([])
    amp_r_results = np.array([])
    cen_r_results = np.array([])
    width_r_results = np.array([])
    skew_r_results = np.array([])
    f_blue_peak_results = np.array([])
    f_red_peak_results = np.array([])
    ratio_peak_results = np.array([])
    f_blue_int_results = np.array([])
    f_red_int_results = np.array([])
    ratio_int_results = np.array([])
    f_results = np.array([])
    fwhm_blue_results = np.array([])
    fwhm_red_results = np.array([])
    vsep_results = np.array([])

    f_total = np.zeros(np.shape(f))

    for i in range(1000):

        # Draw a random Gaussian sample from each flux density bin
        f_mc = np.random.normal(f, n)
        f_total = f_total + f_mc

        # For each emission line used to measure the redshift
        for j, line in enumerate(lines_to_use):

            w_line = w[(w >= windows[j][0]) & (w <= windows[j][1])]
            f_line = f_mc[(w >= windows[j][0]) & (w <= windows[j][1])]
            #n_line = n[(w >= windows[j][0]) & (w <= windows[j][1])]

            _, z = compute_redshift(w_line, f_line, line_dict[line])
            z_results = np.append(z_results, z)

            v = 299792.458 * ((w / (1215.67 * (1 + z))) - 1)

            fcen = compute_fcen(v, f_mc)
            fcen_results = np.append(fcen_results, fcen)

            try:

                #p, cov = fit_peaks(v[(v >= -1000) & (v <= 1000)], f_mc[(v >= -1000) & (v <= 1000)])
                p, cov = fit_peaks(v, f_mc)

                amp_b_results = np.append(amp_b_results, p[0])
                cen_b_results = np.append(cen_b_results, p[1])
                width_b_results = np.append(width_b_results, p[2])
                skew_b_results = np.append(skew_b_results, p[3])

                amp_r_results = np.append(amp_r_results, p[4])
                cen_r_results = np.append(cen_r_results, p[5])
                width_r_results = np.append(width_r_results, p[6])
                skew_r_results = np.append(skew_r_results, p[7])

                if False:

                    fig, ax = plt.subplots(constrained_layout=True)

                    ax.plot(v[(v >= -1000) & (v <= 1000)], f_mc[(v >= -1000) & (v <= 1000)],
                        c='black', ds='steps-mid')
                    ax.plot(v[(v >= -1000) & (v <= 1000)], 
                        skewed_gaussian(v[(v >= -1000) & (v <= 1000)], p[0], p[1], p[2], p[3]),
                        c='blue', ls='dashed')
                    ax.plot(v[(v >= -1000) & (v <= 1000)], 
                        skewed_gaussian(v[(v >= -1000) & (v <= 1000)], p[4], p[5], p[6], p[7]),
                        c='red', ls='dashed')

                    plt.show()

                try:

                    fwhm_blue, fwhm_red = compute_fwhms(v[(v >= -1000) & (v <= 1000)], 
                        skewed_gaussian(v[(v >= -1000) & (v <= 1000)], p[0], p[1], p[2], p[3]),
                        skewed_gaussian(v[(v >= -1000) & (v <= 1000)], p[4], p[5], p[6], p[7]))
                    fwhm_blue_results = np.append(fwhm_blue_results, fwhm_blue)
                    fwhm_red_results = np.append(fwhm_red_results, fwhm_red)
                
                except IndexError:
                    pass

                v_new = np.arange(-1000,1000,1)    

                f_blue_peak, f_red_peak = np.amax(skewed_gaussian(v_new, p[0], p[1], p[2], p[3])), np.amax(skewed_gaussian(v_new, p[4], p[5], p[6], p[7]))
                ratio_peak = f_blue_peak / f_red_peak
                f_blue_peak_results = np.append(f_blue_peak_results, f_blue_peak)
                f_red_peak_results = np.append(f_red_peak_results, f_red_peak)
                ratio_peak_results = np.append(ratio_peak_results, ratio_peak)

                f_blue_int, f_red_int = np.trapz(skewed_gaussian(v[(v >= -1000) & (v <= 1000)], p[0], p[1], p[2], p[3]), w[(v >= -1000) & (v <= 1000)]), np.trapz(skewed_gaussian(v[(v >= -1000) & (v <= 1000)], p[4], p[5], p[6], p[7]), w[(v >= -1000) & (v <= 1000)])
                ratio_int = f_blue_int / f_red_int
                f_blue_int_results = np.append(f_blue_int_results, f_blue_int)
                f_red_int_results = np.append(f_red_int_results, f_red_int)
                ratio_int_results = np.append(ratio_int_results, ratio_int)
                f_results = np.append(f_results, f_blue_int + f_red_int)

                #v_new = np.arange(-1000,1000,1)    
                vsep = compute_vsep(v_new, 
                    skewed_gaussian(v_new, p[0], p[1], p[2], p[3]),
                    skewed_gaussian(v_new, p[4], p[5], p[6], p[7]))

                #vsep = compute_vsep(v[(v >= -1000) & (v <= 1000)],
                #    skewed_gaussian(v[(v >= -1000) & (v <= 1000)], p[0], p[1], p[2], p[3]),
                #    skewed_gaussian(v[(v >= -1000) & (v <= 1000)], p[4], p[5], p[6], p[7]))
                vsep_results = np.append(vsep_results, vsep)

            except RuntimeError:
                pass

    f_total = f_total / 1000
    v = 299792.458 * ((w / (1215.67 * (1 + np.median(z_results)))) - 1)

    figLyaResults, axLyaResults = plt.subplots(figsize=(7,7), constrained_layout=True)

    axLyaResults.set_xlim(-1000,1000)

    figLyaResults.supxlabel(r'Velocity (km s$^{-1}$)')
    figLyaResults.supylabel(r'$F_{\lambda}$ (10$^{-17}$ erg s$^{-1}$ cm$^{-2}$ $\rm{\AA}^{-1}$)')

    axLyaResults.axvline(0.0, c='black', ls='dotted')
    axLyaResults.fill_between(v[(v >= -1100) & (v <= 1100)], (f_total - n)[(v >= -1100) & (v <= 1100)], (f_total + n)[(v >= -1100) & (v <= 1100)], color='gray', alpha=0.25, step='mid')
    axLyaResults.plot(v[(v >= -1100) & (v <= 1100)], f_total[(v >= -1100) & (v <= 1100)], c='black', ds='steps-mid')
    axLyaResults.plot(v[(v >= -1100) & (v <= 1100)], skewed_gaussian(v[(v >= -1100) & (v <= 1100)], np.median(amp_b_results), np.median(cen_b_results), np.median(width_b_results), np.median(skew_b_results)),
        color='blue', ls='dashed')
    axLyaResults.plot(v[(v >= -1100) & (v <= 1100)], skewed_gaussian(v[(v >= -1100) & (v <= 1100)], np.median(amp_r_results), np.median(cen_r_results), np.median(width_r_results), np.median(skew_r_results)),
        color='red', ls='dashed')

    # Annotate the figure with the measurements of the parameters
    at = AnchoredText('FWHM (blue) $ =' + str(round(np.median(fwhm_blue_results))) + '_{-' + str(round(np.median(fwhm_blue_results) - np.percentile(fwhm_blue_results, 16))) + '}^{+' + str(round(np.percentile(fwhm_blue_results, 84) - np.median(fwhm_blue_results))) + '}$ km s$^{-1}$' + '\n'
        'FWHM (red) $ =' + str(round(np.median(fwhm_red_results))) + '_{-' + str(round(np.median(fwhm_red_results) - np.percentile(fwhm_red_results, 16))) + '}^{+' + str(round(np.percentile(fwhm_red_results, 84) - np.median(fwhm_red_results))) + '}$ km s$^{-1}$' + '\n'
        r'$v_{\rm{sep}} =' + str(round(np.median(vsep_results))) + '_{-' + str(round(np.median(vsep_results) - np.percentile(vsep_results, 16))) + '}^{+' + str(round(np.percentile(vsep_results, 84) - np.median(vsep_results))) + '}$ km s$^{-1}$' + '\n'
        r'$F_{\rm{blue}}/F_{\rm{red}} =' + str(round(np.median(ratio_peak_results), 2)) + '_{-' + str(round(np.median(ratio_peak_results) - np.percentile(ratio_peak_results, 16), 2)) + '}^{+' + str(round(np.percentile(ratio_peak_results, 84) - np.median(ratio_peak_results), 2)) + '}$' + '\n'
        r'$f_{\rm{cen}} =' + str(round(np.median(fcen_results))) + '_{-' + str(round(np.median(fcen_results) - np.percentile(fcen_results, 16))) + '}^{+' + str(round(np.percentile(fcen_results, 84) - np.median(fcen_results))) + '}$ %' + '\n' + '\n'
        r'$z =' + str(round(np.median(z_results), 4)) + '_{-' + str(round(np.median(z_results) - np.percentile(z_results, 16), 4)) + '}^{+' + str(round(np.percentile(z_results, 84) - np.median(z_results), 4)) + '}$' + '\n' + '\n'
        r'$F_{\rm{Ly}\alpha} =' + str(round(np.median(f_results))) + '_{-' + str(round(np.median(f_results) - np.percentile(f_results, 16))) + '}^{+' + str(round(np.percentile(f_results, 84) - np.median(f_results))) + '}$ 10$^{-17}$ erg s$^{-1}$ cm$^{-2}$' + '\n'
        r'$F_{\rm{blue}}^{\rm{int}} =' + str(round(np.median(f_blue_int_results))) + '_{-' + str(round(np.median(f_blue_int_results) - np.percentile(f_blue_int_results, 16))) + '}^{+' + str(round(np.percentile(f_blue_int_results, 84) - np.median(f_blue_int_results))) + '}$ 10$^{-17}$ erg s$^{-1}$ cm$^{-2}$' + '\n'
        r'$F_{\rm{red}}^{\rm{int}} =' + str(round(np.median(f_red_int_results))) + '_{-' + str(round(np.median(f_red_int_results) - np.percentile(f_red_int_results, 16))) + '}^{+' + str(round(np.percentile(f_red_int_results, 84) - np.median(f_red_int_results))) + '}$ 10$^{-17}$ erg s$^{-1}$ cm$^{-2}$',
        loc='upper left', frameon=False)
    axLyaResults.add_artist(at)

    figLyaResults.savefig(figs + '/lya_and_redshift_measurements/' + lae + '.pdf', bbox_inches='tight')

    print('FWHM (blue) =', np.median(fwhm_blue_results), np.median(fwhm_blue_results) - np.percentile(fwhm_blue_results, 16), np.percentile(fwhm_blue_results, 84) - np.median(fwhm_blue_results))
    print('FWHM (red) =', np.median(fwhm_red_results), np.median(fwhm_red_results) - np.percentile(fwhm_red_results, 16), np.percentile(fwhm_red_results, 84) - np.median(fwhm_red_results))
    print('vsep =', np.median(vsep_results), np.median(vsep_results) - np.percentile(vsep_results, 16), np.percentile(vsep_results, 84) - np.median(vsep_results))
    print('F_blue / F_red =', np.median(ratio_peak_results), np.median(ratio_peak_results) - np.percentile(ratio_peak_results, 16), np.percentile(ratio_peak_results, 84) - np.median(ratio_peak_results))
    print('fcen =', np.median(fcen_results), np.median(fcen_results) - np.percentile(fcen_results, 16), np.percentile(fcen_results, 84) - np.median(fcen_results))

    print('z =', np.median(z_results), np.median(z_results) - np.percentile(z_results, 16), np.percentile(z_results, 84) - np.median(z_results))

    print('FLya =', np.median(f_results), np.median(f_results) - np.percentile(f_results, 16), np.percentile(f_results, 84) - np.median(f_results))
    print('FLyaBlue =', np.median(f_blue_int_results), np.median(f_blue_int_results) - np.percentile(f_blue_int_results, 16), np.percentile(f_blue_int_results, 84) - np.median(f_blue_int_results))
    print('FLyaRed =', np.median(f_red_int_results), np.median(f_red_int_results) - np.percentile(f_red_int_results, 16), np.percentile(f_red_int_results, 84) - np.median(f_red_int_results))

    print('samples: ', len(vsep_results))

    #plt.hist(vsep_results, bins=30)
            

In [4]:
compute_lya_properties(37, 
    ['OIII] 1666'],
    [[5336,5343]])

5359-55953-51


IndexError: list index out of range